### To-Do
- csv file c1(csv file id) and c2 (meta file location)
- extract meta-data from file
- enriched csv file with metadata
- create EML file from metadata xml file

In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_colwidth=0

In [2]:
def extract_metadata(xml_file):
    pass
    

def read_path(csv_file):
    df = pd.read_csv(csv_file, encoding='utf-8')
    return df

def set_meta_file_path(path):
    path = path.split('images/')[0]
    return path+'meta.xml'
    


In [4]:
df = read_path('./local_data/imageId_metapath.csv')
df['MetaFile'] = df['Path'].apply(set_meta_file_path)
df.sort_values(by='MetaFile',inplace=True)

In [5]:
metafile_metadata_dict = {i:{} for i in df['MetaFile'].unique().tolist()}
metafile_metadata_dict

{'unziped_02_dec/tx_7397/28978/meta.xml': {},
 'unziped_02_dec/tx_7407/28981/meta.xml': {},
 'unziped_02_dec/tx_7423/28986/meta.xml': {},
 'unziped_02_dec/tx_7426/28987/meta.xml': {},
 'unziped_02_dec/tx_7447/28988/meta.xml': {},
 'unziped_02_dec/tx_7454/28990/meta.xml': {},
 'unziped_02_dec/tx_7462/28991/meta.xml': {},
 'unziped_02_dec/tx_7464/28992/meta.xml': {},
 'unziped_02_dec/tx_7465/28993/meta.xml': {},
 'unziped_02_dec/tx_7467/28994/meta.xml': {},
 'unziped_02_dec/tx_7483/28997/meta.xml': {},
 'unziped_02_dec/tx_7484/28998/meta.xml': {},
 'unziped_02_dec/tx_7487/29001/meta.xml': {},
 'unziped_02_dec/tx_7493/29002/meta.xml': {},
 'unziped_02_dec/tx_7498/29003/meta.xml': {},
 'unziped_02_dec/tx_7518/29005/meta.xml': {},
 'unziped_02_dec/tx_7533/29008/meta.xml': {},
 'unziped_02_dec/tx_7549/29009/meta.xml': {},
 'unziped_02_dec/tx_7595/29013/meta.xml': {},
 'unziped_02_dec/tx_7608/29014/meta.xml': {},
 'unziped_02_dec/tx_7609/29015/meta.xml': {},
 'unziped_02_dec/tx_7636/29016/met

In [38]:
"""('AdditionalNotes', 'Sonderabdruck aus, Decheniana, Verhandlungen des Naturhistorischen Vereins der Rheinlande und Westfalens, Band 101 A B'),
   'PlaceOfPublication', 'Bonn',
   'PublisherName', 'Hch. Ludwig',
   'TxMainLanguage', 'german',
   'TxType', 'book',
   'TxSpeciesData', 'yes',
   'TxFormat', 'A4',
   'SizeSourcePrint', 'Seiten 131-214g, L1-L18',
   'Illustration', 'Diagramme',
   'pathimagefiles', './images/',
   '_representative', '1',
   'Author', {'type', 'person',
             'lastName', 'Budde',
             'firstName', 'Hermann',
             'authorityID', 'gnd',
             'authorityURI', 'http,//d-nb.info/gnd/',
             'authorityValue', 1028522606,
             'displayName', 'Budde, Hermann'}}"""


mappping_metaxml_to_eml_list = [('PublicationYear', 'eventDate'), 
                                 ('PublicationYear', 'year'), 
                                  ('PublicationYear', 'publicationYear'),
                                 ('TitleDocMain', 'title'),('TitleDocMain', 'publicationTitle'), 
                                  ('TxID','catalogNumber'), ('DocLanguage', 'language'),
                                  ('singleDigCollection', 'collectionCode'), ('TxAuthor','publicationAuthors'),
                                  ('singleDigCollection', 'parentCollectionIdentifier'), 
                                 ('CatalogIDDigital', 'collectionIdentifier'), ('authorityURI', 'authorityURI'), 
                                  ('authorityValue', 'authorityValue'),]

mappping_metaxml_to_eml_dict = {i[1]:i[0] for i in mappping_metaxml_to_eml_list}

print (mappping_metaxml_to_eml_dict)

def remove_null_values_from_nested_dict(nested_dict):
    if not isinstance(nested_dict, dict):
        return nested_dict

    new_dict = {}
    for key, value in nested_dict.items():
        if value is not None and not isinstance(value, dict) and not value!=value:
            if type(value)==float:
                value = int(value)
            new_dict[key] = value
        elif isinstance(value, dict):
            if type(value)==float:
                value = int(value)
            new_dict[key] = remove_null_values_from_nested_dict(value)

    return new_dict


for key, value in metafile_metadata_dict.items():
    # ToDo -------- Temporary Path -------------
    df_meta = pd.read_xml('./local_data/metadata/meta.xml', xpath="//goobi:metadata",
                      namespaces={"goobi":"http://meta.goobi.org/v1.5.1/"})
    df_meta.set_index(keys='name', inplace=True)
    df_meta = df_meta[~df_meta.index.duplicated()]
    dict_metadata = df_meta.to_dict()
    new_dict = {}
    new_dict['metadata'] = remove_null_values_from_nested_dict(df_meta['metadata'].to_dict())
    new_dict['Author'] = remove_null_values_from_nested_dict(df_meta.loc['Author'].to_dict())
    # metafile_metadata_dict[key]=new_dict

    new_dict1 = {}
    for k, v in new_dict.items():
        # print (new_dict)
        k1_v1_dict = {k1:v1 for k1, v1 in new_dict[k].items() if k1 in mappping_metaxml_to_eml_dict.values()}
        # print(k1_v1_dict)
        new_dict1.update({k2:k1_v1_dict[k1] for k2, k1 in mappping_metaxml_to_eml_dict.items() if k1 in k1_v1_dict.keys()})
    metafile_metadata_dict[key] = new_dict1
    # Todo -------- Remove Break ----------------
    break

df_meta = pd.DataFrame.from_dict(metafile_metadata_dict, orient='index' ).reset_index()
## temporary testing
## df_meta.loc[0,'index']='unziped_02_dec/tx_8369/29090/meta.xml'
df_meta

{'eventDate': 'PublicationYear', 'year': 'PublicationYear', 'publicationYear': 'PublicationYear', 'title': 'TitleDocMain', 'publicationTitle': 'TitleDocMain', 'catalogNumber': 'TxID', 'language': 'DocLanguage', 'collectionCode': 'singleDigCollection', 'publicationAuthors': 'TxAuthor', 'parentCollectionIdentifier': 'singleDigCollection', 'collectionIdentifier': 'CatalogIDDigital', 'authorityURI': 'authorityURI', 'authorityValue': 'authorityValue'}


,index,eventDate,year,publicationYear,title,publicationTitle,catalogNumber,language,collectionCode,publicationAuthors,parentCollectionIdentifier,collectionIdentifier,authorityURI,authorityValue
0,unziped_02_dec/tx_7397/28978/meta.xml,1942,1942,1942,Die Algenflora Westfalens und der angrenzenden Gebiete,Die Algenflora Westfalens und der angrenzenden Gebiete,7487,ger,Tuexen,Hermann Budde,Tuexen,tuexen_7487,http://d-nb.info/gnd/,1028522606


In [39]:
df.merge(df_meta, left_on='MetaFile', right_on='index', how='left')

,Image_Id,Path,MetaFile,index,eventDate,year,publicationYear,title,publicationTitle,catalogNumber,language,collectionCode,publicationAuthors,parentCollectionIdentifier,collectionIdentifier,authorityURI,authorityValue
0,453,unziped_02_dec/tx_7397/28978/images/Tuexen_7397_tif/00000010.tif,unziped_02_dec/tx_7397/28978/meta.xml,unziped_02_dec/tx_7397/28978/meta.xml,1942,1942,1942,Die Algenflora Westfalens und der angrenzenden Gebiete,Die Algenflora Westfalens und der angrenzenden Gebiete,7487,ger,Tuexen,Hermann Budde,Tuexen,tuexen_7487,http://d-nb.info/gnd/,1.028523e+09
1,640,unziped_02_dec/tx_7407/28981/images/Tuexen_7407_tif/00000016.tif,unziped_02_dec/tx_7407/28981/meta.xml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,394,unziped_02_dec/tx_7423/28986/images/Tuexen_7423_tif/00000105.tif,unziped_02_dec/tx_7423/28986/meta.xml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,393,unziped_02_dec/tx_7423/28986/images/Tuexen_7423_tif/00000059.tif,unziped_02_dec/tx_7423/28986/meta.xml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,392,unziped_02_dec/tx_7423/28986/images/Tuexen_7423_tif/00000061.tif,unziped_02_dec/tx_7423/28986/meta.xml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,136,unziped_02_dec/tx_8338/29088/images/Tuexen_8338_tif/00000005.tif,unziped_02_dec/tx_8338/29088/meta.xml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
680,67,unziped_02_dec/tx_8369/29090/images/Tuexen_8369_tif/00000014.tif,unziped_02_dec/tx_8369/29090/meta.xml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
681,66,unziped_02_dec/tx_8369/29090/images/Tuexen_8369_tif/00000009.tif,unziped_02_dec/tx_8369/29090/meta.xml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
682,65,unziped_02_dec/tx_8369/29090/images/Tuexen_8369_tif/00000006.tif,unziped_02_dec/tx_8369/29090/meta.xml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
